# [Exploration 13] 어제 오른 내 주식, 과연 내일은?
## 프로젝트 설명 
---
ARIMA를 이용해 시계열 데이터 중 주식 데이터를 예측해본다.


## 프로젝트 정리     
---        
1. 데이터 준비
2. 데이터 전처리
3. 시계열 데이터 안정성 분석
4. 데이터셋 생성
5. 적정 ARIMA 모수 찾기
6. ARIMA 모델 훈련과 테스트 
7. 다른 주식 종목 예측해 보기


## 프로젝트 목표
---



## 01. 데이터 준비
- [Yahoo Finance](https://finance.yahoo.com/)사이트를 이용해 주식 데이터 다운로드합니다.
- 최근 관심있었던 [S&P 500](https://finance.yahoo.com/quote/%5EGSPC/history?p=%5EGSPC)로 진행하겠습니다.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
dataset_filepath = os.getenv('HOME')+'/aiffel/stock_prediction/data/^GSPC.csv'
df = pd.read_csv(dataset_filepath)
print(type(df))
df.head()

<class 'pandas.core.frame.DataFrame'>


,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-03-03,3096.459961,3136.719971,2976.629883,3003.370117,3003.370117,6355940000
1,2020-03-04,3045.750000,3130.969971,3034.379883,3130.120117,3130.120117,5035480000
2,2020-03-05,3075.699951,3083.040039,2999.830078,3023.939941,3023.939941,5575550000
3,2020-03-06,2954.199951,2985.929932,2901.540039,2972.370117,2972.370117,6552140000
4,2020-03-09,2863.889893,2863.889893,2734.429932,2746.560059,2746.560059,8423050000


- 여러 데이터가 있지만 종가만 사용하겠습니다.
    * 종가(Close) : 주식 시장이 마감했을 때의 주식 가격
- dataframe 형태의 파일을 Series 형태로 변경하겠습니다.

In [5]:
df = pd.read_csv(dataset_filepath, index_col='Date', parse_dates=True)
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-03-03,3096.459961,3136.719971,2976.629883,3003.370117,3003.370117,6355940000
2020-03-04,3045.750000,3130.969971,3034.379883,3130.120117,3130.120117,5035480000
2020-03-05,3075.699951,3083.040039,2999.830078,3023.939941,3023.939941,5575550000
2020-03-06,2954.199951,2985.929932,2901.540039,2972.370117,2972.370117,6552140000
2020-03-09,2863.889893,2863.889893,2734.429932,2746.560059,2746.560059,8423050000


In [6]:
ts_snp = df['Close']
print(type(ts_snp))
ts_snp.head()

<class 'pandas.core.series.Series'>


Date
2020-03-03    3003.370117
2020-03-04    3130.120117
2020-03-05    3023.939941
2020-03-06    2972.370117
2020-03-09    2746.560059
Name: Close, dtype: float64

# 루브릭 평가
---
1. **GAN의 두 모델 구조를 통해 이미지를 성공적으로 생성하였다.**   

    - 레이어를 수정하는 것이 가장 큰 도움이 되었다.
    - G, D의 아웃풋을 똑같게 하는 것이 아니라 다르게 조정하니까 확실히 개선이 되었다.


2. **생성 이미지 시각화 및 학습 그래프를 통해 GAN 학습이 바르게 진행되었음을 입증하였다.**

    - 마지막 학습 그래프를 보면 fake accuracy가 0.5에 가깝게 가고 있는 것을 확인할 수 있었다. 
     
     
3. **추가적인 GAN 모델구조 혹은 학습과정 개선 아이디어를 제안하고 이를 적용하였다.** 

    - 그래프를 연달아 보면서 품질이 개선되는 것을 확인할 수 있었다.
    - 만들어진 샘플들이 변화하는 것을 봐도 확실히 개선되었다.
    
    
# 회고
---
## 어려웠던 점 
- 모델
    - 사실 모델의 개략적인 메커니즘만 노드에서 설명이 되어있기 때문에, 성능을 개선하기 어려웠다.
    - 실제 모델의 수식 등을 이해하면 성능 개선 부분이 수월했을 것 같다.
    
## 알게된 점 
- encoder, decoder와 다르다!
    - 처음엔 G와 D의 레이어 필터 수나 output shape을 동일하게 해야하는 줄 알았는데, 아니었다. 
    - G는 이미지를 생성하는 역할, D는 생성된 이미지를 판별한다.
    - D는 이미지를 판별해야하기 때문에 더 촘촘한 레이어가 필요하고, G는 처음엔 넓은 범위(edge)를 먼저 생성해도 된다고 생각한다.
    - 이런 식으로 정보가 많이 필요하면 레이어를 촘촘하게 구성해서 파라미터를 늘이고, 등등의 직관을 사용해 구성하니 훨씬 생성이 잘 되었다.
    
## 더 공부할 점
- 다른 GAN모델
    - 다른 모델도 적용해봐야겠다.
- accuracy가 왜 batch마다 상이할까?
    - 다른 알고리즘은 학습횟수가 증가함에 따라 지표가 점층적으로 목표점으로 수렴했는데, GAN은 batch마다 너무 달랐다.
    - 내가 잘못 학습한 줄 알았는데, 구글링해보니 다들 이런 모양의 그래프가 나왔다.
    - 어떤 블로그에서는 이런 점때문인지 early stopping을 하는 것이 좋지 않다고 했는데, 이유를 공부해야겠다.

## 다짐
- 어떤 모델을 공부할 때, 논문을 읽어봐야겠다. 다른 사람들의 글도 충분히 이해가 가고 도움이 되지만, 직접 튜닝을 해서 원하는 결과물을 도출하고자 할 때는 연구자만큼 이해해야 가능한 것 같다. 